Copyright 2021 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

**Instructions**: the notebook linked to in the go link above is view-only.  Make your own personal copy by selecting "File > Save a copy in Drive", because the exercises are to modify the code snippets.

To execute code snippets, click the "play" button on the left edge of each code box.  You can also use the "runtime" menu above if you need to run many code boxes at once.

# Introduction

For this session, we will take a small detour and explore the theoretical underpinnings of the RSA cryptosystem.  This draws more upon elementary number theory than abstract algebra, but these subjects are closely related.

A word of warning: while we will describe the key idea that underlies RSA, the actual implementation of RSA (or any modern cryptosystem) is much more challenging, as implementation requires careful thought to ensure that weaknesses do not leak into the implementation.  We will describe one example of a simple implementation flaw found in the wild 10 years ago, just to give a taste of how implementation details can introduce weaknesses not present in the theoretical description of the cryptosystem.  **tl;dr**: don't roll your own crypto!

# A simplistic overview of cryptography

References: https://en.wikipedia.org/wiki/Caesar_cipher, https://en.wikipedia.org/wiki/Symmetric-key_algorithm

Broadly speaking, **cryptography** is the study of methods of securely transmitting information between parties (typically two parties) in such a way as to keep the contents private from other parties.  As we all know, cryptography is a critical foundation of the modern Internet, but even before the modern computer era, cryptography had many practical applications, especially in warfare.



One of the earliest historically documented uses of cryptography was Julius Caesar's use of the so-called *Caesar* or *shift* cipher.  Given a message encoded using the letters A through Z, a message is encrypted by shifting each character by a certain fixed number of characters, rotating through the alphabet if necessary.  For example, Caesar supposedly used this cipher to encrypt messages by shifting each character three positions down the alphabet; ie, "A" -> "D", "B" -> "E", ..., "Z" -> "C".  

A message like 

`HELLO`

then becomes

`KHOOR`

The recipient of this message then decrypts by simply reversing the shift.

In [ ]:
# Exercise: Implement the shift cipher for strings composed of capital letters.
# Input: a string whose characters are either capital letters or spaces, and
# a shift length n.
# Output: the encrypted string
def rot_n(plaintext, n):
  # TODO: Your implementation here
  pass

This obviously generalizes to any shift cipher where the shift length 3 can be changed to any number between 1 and 25.  Even though this method of encryption is trivially easy to beat by modern standards, it might have been sufficiently strong to protect communications from Rome's enemies in ancient times.

A key observation in this particular method of encryption/decryption is that the information required to encrypt (the shift length) is essentially identical to the information required to decrypt (the inverse of the shift length).  In other words, using the Caesar cipher, **you cannot encrypt a message without also knowing how to decrypt it**.   In general, an encryption algorithm in which knowledge of how to encrypt is equal to (or an easy transformation of) knowledge of how to decrypt is called a **symmetric-key algorithm**.

For much of human history (including through World War II), essentially all encryption algorithms were symmetric-key algorithms, although they were far more elaborate than shift ciphers.  Capturing codebooks and breaking enemy encryption protocols were an important part of military strategy (for example, the Americans gained a critical advantage over the Japanese at the Battle of Midway when they were mostly able to break encryption on Japanese communications).

One major shortcoming of a symmetric-key algorithm is that it requires sender and recipient of message to agree beforehand on the shared encryption/decryption key, which historically required agreeing on and distributing a shared physical secret, such as a codebook or passphrase.  This key needs to be kept secret by all parties involved; as soon as the (secret) key is leaked publicly, all encrypted messages are compromised.

# Modern Cryptography: RSA, the first public-key cryptosystem

Reference: https://en.wikipedia.org/wiki/RSA_(cryptosystem)

Even as progressively more elaborate symmetric-key encryption schemes were developed, they faced the fundamental problem of distributing and keeping secure that private encryption/decryption key.  A series of major breakthroughs occurred in the mid 1970s, when researchers discovered (in some cases, re-discovered) methods of distributing shared secrets over public channels securely (the Diffie-Hellman key exchange) and an entirely new encryption algorithm (RSA) which, for the first time, did not require the use of a shared key for encryption and decryption.

The key insight was the discovery of mathematical operations which were computationally practical to perform, but computationally difficult (de facto impossible) to reverse.  In RSA, the easy computational operation, which encrypts a message, is performed using a *public key*, which is known to the world at large.  However, decrypting requires knowledge of a *private key*, which is only held by the party receiving messages, and is otherwise computationally infeasible to carry out.

The RSA cryptosystem is built on the idea that **modular exponentiation is computationally cheap**, but reversing this operation ("taking kth roots mod n") is, as far as we know, computationally expensive.

## Fast modular exponentiation

Reference: https://en.wikipedia.org/wiki/Modular_exponentiation

As a pre-requisite, we will describe a naive, but fast, modular exponentiation algorithm.  Concretely, we want to compute the remainder of $a^{k}$ upon division by some fixed number $n$.  For example, what is $31^{382910406} \bmod 145957507571$?

First, note that computing $a^{k}$ as an integer can actually be computationally expensive because the number of digits increases as you do more and more multiplications.  So we will avoid first computing $a^{k}$ and then taking a Euclidean division by $n$.  Instead, we will do all multiplications mod n to keep the size of all numbers below $n^{2}$.

With this in mind, the most naive exponentiation algorithm would be:




In [ ]:
# Non-optimally computes remainder of a^k divided by n.
def slow_modular_expt(a, k, n):
  ans = 1
  for i in range (k):
    ans = (ans * a) % n
  return ans

In [ ]:
slow_modular_expt(3, 372010, 24)

Python actually has a modular exponentiation function built-in; you can check your work by using the pow function:

In [ ]:
slow_modular_expt(37, 93603, 1839) == pow(37, 93603, 1839)

This algorithm is  $O(k * (\log n)^{2})$; for large values of $k$ this may be too slow.

The clever trick to reduce the number of multiplications to $O(\log k)$ is to use the binary representation of $k$ together with *successive squares* of $a$; that is, $a, a^{2}, a^{4}, a^{8}, \ldots$. For example, to compute $a^{94} \bmod 173$, first calculate the binary representation of ${94}$, which is $1011110$.  Then

$a^{94} = a^{64} * a^{16} * a^{8} * a^{4} * a^{2}$.

It takes about $\log k$ multiplications to compute each of these successive squares, and up to another $\log k$ multiplications to multiply the appropriate squares together.

In [ ]:
# Exercise: Implement fast modular exponentiation.
# Input: positive integers a, k, and n.
# Output: remainder of a^k divided by n
def fast_modular_expt(a, k, n):
  # TODO: implement fast modular exponentiation
  pass

You can test your function by comparing its output to the Python pow function, as done in the example above.

In [ ]:
fast_modular_expt(37, 93603, 1839) == pow(37, 93603, 1839)

## A description of RSA

OK.  With the pre-requisites out of the way, we can now describe how to encrypt and decrypt messages using RSA.

The first part of the algorithm is the generation of a public/private key pair by the eventual recipient of messages.  For example, suppose Alice wants to receive messages from Bob.  Alice will do the following to create a public/private key pair:

* Select two "large" primes p and q.  (How we actually select p and q is a very rich topic in its own right.)
* Compute $n = p * q$ and $\varphi(n) = (p-1) * (q-1)$.  
* Select some integer $e$ with $2 < e < n-1$ satisfying $\gcd(e, \varphi(n)) = 1$.  For example, just repeatedly choose random values of $e$ until Alice finds one with $\gcd(e, \varphi(n)) = 1$.
* Compute an integer $d$ satisfying $de \equiv 1 \bmod \varphi(n)$ and $0 < d < \varphi(n)$; in other words, compute a modular inverse for $e$ $\bmod \varphi(n)$.
* Publish $(n, e)$ as the public key, and keep $d$ as the private key.

Now suppose Bob wants to send Alice a message $m$.  We assume $m$ is an integer satisfying $1 < m < n$.  Bob encrypts using the public key by computing, using fast modular exponentiation,

$m^{e} \equiv c \bmod n$

$c$ is the *ciphertext*, or encrypted message, that Bob sends to Alice, possibly over an insecure channel with eavesdroppers (like the Internet).

Alice decrypts $c$ by using the private key $d$, by computing $c^{d} \bmod n$. Notice that $c^{d} \equiv m^{ed} \bmod n$.  If $\gcd(m, n) = 1$, then the Fermat-Euler theorem says that $m^{\varphi(n)} \equiv 1 \bmod n$, so that $m^{ed} = m^{1 + k \cdot \varphi(n)} \equiv m \bmod n$.



**Exercise**.  Check that the restriction $\gcd(m, n) = 1$ is not necessary above; eg, that even if $\gcd(m, n) > 1$, then $m^{ed} \equiv m \bmod n$.

## A basic RSA example



Let's work through a basic example to illustrate how RSA works.  Suppose you want to generate a public/private key pair.  You don't care too much about security, so you pick primes $p, q$ that only have a few digits, say (if you want, feel free to replace with your own primes):

In [ ]:
p, q = 131071, 324697

Compute $n = p * q$ and $\varphi(n) = (p-1) * (q-1)$: 

In [ ]:
n, varphi = p * q, (p-1) * (q-1)
print(f"n = {n}, varphi = {varphi}")

n = 42558360487, varphi = 42557904720


Randomly select $e$ satisfying $\gcd(e, \varphi) = 1$.  You can either use your own implementation of gcd from last week, or if you want to use a standard library implementation, import math and use math.gcd from the Python standard library:

In [ ]:
import math

To generate random numbers, you can use the python random library.  Since this is just a toy exercise, we don't need cryptographic-strength random number generators.

In [ ]:
import random

In [ ]:
rng = random.SystemRandom()

# Exercise: Implement random selection of e, subject to gcd(e, varphi) == 1.

# to randomly generate an integer between 3 and n-2, use:
e = rng.randint(3, n-2)
print(math.gcd(e, varphi)) 
# TODO: fill in loop and test for gcd(e, varphi) == 1.
pass
print(f"e = {e}")

1
e = 39246239449


With whatever value of $e$ you computed in the previous step, find its modular inverse mod $\varphi(n)$.  Use your implementation of the extended Euclidean algorithm from our colab on that topic.

In [ ]:
def bezout(a, b):
  if a == 0 or b == 0:
    return ValueError('need both inputs to be non-zero')
  sgna, sgnb = a // abs(a), b // abs(b)
  a, b = abs(a), abs(b)
  if a < b:
    x, y, g = bezout(b, a)
    return y, x, g
  xi, yi = 1, 0
  xnext, ynext = 0, 1
  rold, r = a, b
  while r != 0:
    q = rold // r
    rold, r = r, rold % r
    xi, yi, xnext, ynext = xnext, ynext, xi - q * xnext, yi - q * ynext
  return sgna * xi, sgnb * yi, rold

In [ ]:
d, _, _ = bezout(e, varphi)
d = d % varphi # Want to make sure d is positive
print(f"d = {d}")

d = 23384065609


Let's just verify that $d, e$ really are modular inverses...

In [ ]:
(d * e) % varphi == 1

True

The public key is the ordered pair $(n, e)$.  Suppose someone wants to encrypt the message $m = 123456$.  Encrypt this by computing $m^{e} \bmod n$:

In [ ]:
m = 123456
# Exercise: encrypt m using (n, e) obtained from above
c = pow(m, e, n)  # TODO: calculate c
print(c)


24746566528


Check that this can indeed be decrypted with the private key:

In [ ]:
# Exercise: Replace TODO with the decrypting expression and check that it equals
# the original message m.
pow(c, d, n) == m

True

You should recover $m$.  That's how RSA works!

## What the security of RSA relies on

Now that we have a description of how RSA works and worked through a toy example, let's discuss why it is considered secure.

For an attacker to recover $m$ from just $n, e$ and $m^{e} \bmod n$, it would be sufficient for the attacker to be able to do one of the following:

* factor $n$ and/or compute $\varphi(n)$.  As soon as an attacker knows $p$ or $q$, they can just compute $d$ because they can compute the modular inverse of $e \bmod \varphi(n)$.  Note that for semiprimes $n = pq$, factoring $n$ is essentially equivalent to computing $\varphi(n) = pq - p - q + 1 = n - p - q + 1$.
* come up with some way of taking "eth roots mod n": in other words, efficiently extract $m$ from $m^{e} \bmod n$, given known $e, n$.  This is called the **RSA problem** and is evidently at most as hard as integer factorization.  Whether it is equivalently hard (up to polynomial-time equivalence) is an open question.

The most obvious route to compromising RSA is through an efficient factorization algorithm, or at least an efficient factorization algorithm for semiprimes (although as best we can tell semiprimes are the hardest case).  But even the best known integer factorization algorithms are still fairly inefficient; for example, generic 2048-bit semiprimes are still out of reach for modern computers.

Surprisingly enough there is no consensus for the runtime of an asymptotically optimal factorization algorithm; there isn't even expert consensus on whether factorization might be polynomial time or not.  In contrast, primality testing *is* polynomial time!

## Factorization compromises RSA

In this section, we'll work through a concrete example where you will use integer factorization to break RSA.  The numbers in this example are chosen to be of an appropriately small size so that factorization is possible using naive techniques, but of course in practice the primes in RSA are chosen to be sufficiently large to be out of reach of current hardware and software (or so we believe).

You are eavesdropping on the Internet and see the public key (10511327576641, 985306163) being used to transmit the encrypted message c = 334941853134.  Decrypt c.

The most straightforward attempt is to factor $n = 10511327576641$.  This number is 14 digits long, so a straightforward "brute-force" approach to find a factor should work.

In [ ]:
# Exercise: write code to find a non-trivial factor of n.
def factor(n):
  # Implement a function to find a non-trivial factor of n.
  pass

In [ ]:
# Use n2, e2, c2 to not collide with numbers from previous section.
n2, e2, c2 = 10511327576641, 9718006108929, 334941853134

In [ ]:
factor(n2)

Use this information to decrypt $c$.

In [ ]:
# Exercise: Compute varphi and then d, and then compute c^d mod n$.
varphi2 = TODO
d2 = TODO
print(pow(c2, d2, n2))


What did you get?  (I was too lazy to make it a clever message.)

## A real life weakness: poor prime number generation

Reference: https://eprint.iacr.org/2012/064.pdf

As an illustration of how weaknesses can leak into any cryptosystem, even if they are theoretically secure, we describe a weakness discovered in the field in 2012, impacting about 0.2% of RSA moduli studied by the authors of the paper above.

The idea is very simple: in the key generation phase of RSA, you need to select two randomly generated distinct large primes $p, q$.  What if the random number generator that is used is not so good, and sometimes recycles the same primes more often than should occur if a strong random number generator is used?

Suppose two distinct public keys $n_{1}$ and $n_{2}$ (we ignore $e$) share a prime factor $p$ (note: it is also possible for two public keys to be identical, in which case the owner of the private key can read the other party's messages!).  Then $\gcd(n_{1}, n_{2}) = p$.  But recall - gcd can be computed quickly!  So this suggests a brute force attack to simply compute pairwise gcds of all RSA public keys out in public.  The authors of the above study did this for ~6.6 million keys, and discovered that $\gcd > 1$ for about 0.2% of said keys.  This means each of those 0.2% of keys actually provided no security whatsoever, and any messages sent with those keys could be read by anybody on the Internet willing to do this fairly elementary work.

The moral of the story is that there may be all sorts of clever ways to attack weaknesses in the implementation of a cryptosystem, even if you theoretically believe it is secure.  This is a concrete example of the importance of strong random number generation in cryptography!

In [ ]:
# Exercise: carry out the above analysis for the following semiprimes
# (that is, factor at least one such semiprime by doing a pairwise gcd
# calculation)
moduli = [317958029403792702239179, 52176858368729257359169667, 1193645648690415035468090429, 2487412961316445490875380541, 1179171486914945257143171401]
# TODO: some code which uses gcds to quickly find a factorization of at least two of the above numbers.

## RSA in real life

Reference: https://en.wikipedia.org/wiki/Transport_Layer_Security

 

In real-life, RSA is typically used as part of a *hybrid cryptosystem*, which uses a public-key algorithm like RSA or Diffie-Hellman to securely exchange a shared key for use in a symmetric-key algorithm, like AES.  This is done for performance reasons, because while RSA is reasonably performant, it is not as performant as cutting edge symmetric key algorithms.  

The key-length as of 2021 is usually 2048 bits, as this is a length which is considered secure for the foreseeable future.  When RSA was first introduced in the 1970s, 512-bit keys were commonly used.  Modern computers can break that in several hours.  More recently, academic researchers were able to factor <a href="https://en.wikipedia.org/wiki/RSA_numbers#RSA-250">RSA-250</a>, a 829-bit semiprime, in 2020, which suggests that 1024 bit keys are at the boundary of becoming insecure (if not from "regular" people, than perhaps from nation-states).

There are other aspects to cryptography besides encrypting and decrypting messages.  For example, RSA can be used to generate digital signatures, which provide evidence that the sender of a message really is who they claim to be.  Cryptography is a rich field in its own right, and is much more than just the theoretical underpinnings from number theory, and there is much to learn beyond what we've discussed here.

# A word about elliptic curve cryptography

References: https://en.wikipedia.org/wiki/Elliptic-curve_cryptography, https://en.wikipedia.org/wiki/Elliptic_curve

We will conclude by briefly discussing elliptic curve cryptography and draws upon ideas discussed throughout this entire course.  



The Diffie-Hellman key exchange protocol, which we did not discuss, is based on the group operation in $(\Bbb Z / p\Bbb Z)^{*}$, and its security is based on the difficulty of something called the *discrete logarithm problem*, which is the question of solving for $x$ in the equation $a^{x} \equiv b \bmod p$, where $a, b$ are fixed and $p$ is prime.  

In the 1980s, the mathematicians Neal Koblitz and Victor Miller independently suggested generalizing this idea, where instead of using $(\Bbb Z / p\Bbb Z)^{*}$, one instead uses the group operation on a different group called an *elliptic curve*.

Elliptic curves are extremely rich objects from number theory and algebraic geometry, which can be described in several different yet ultimately equivalent ways.  The most concrete description of an elliptic curve is as the set of solutions $(x, y)$ to the equation

$y^{2} = x^{3} + Ax + B$,

for some fixed $A, B$, which are usually integers.  You can consider solutions over real or complex numbers, rational numbers, or, in the case of elliptic curve cryptography, over finite fields (of which $\Bbb Z / p\Bbb Z$ is one example).  

The solution set over any of the above fields form a group, with a group law given by a particular geometric operation we do not explain here. One can then describe an <a href="https://en.wikipedia.org/wiki/Elliptic-curve_Diffie%E2%80%93Hellman">analogous Diffie-Hellman protocol</a> for a particular elliptic curve. In analogy to the original DH protocol, its security is premised on the difficulty of the corresponding discrete log problem for the group operation on an elliptic curve.

One major advantage for using elliptic curves over RSA is performance.  It turns out that one can obtain equivalent security (as best we know given current knowledge and technology) with substantially shorter keys, which makes computations faster.  This comes at the cost of a more complex, non-trivial group law, but this is a cost paid by the implementers of the algorithms, not the users of the algorithm.

Elliptic curve cryptography started to appear in real-world applications in the mid 2000s, and is now commonly used, for example in OpenSSL or a wide variety of encrypted messaging apps. 

Elliptic curves are central objects in modern number theory, and their practical application to cryptography is a wonderful illustration of the principle that modern theoretical mathematics, while often esoteric, can have profound real-life applications decades or even centuries after their discovery.